In [1]:
import pandas as pd 
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import folium
from geojson import Feature, FeatureCollection, Point
import json
from scipy.spatial import ConvexHull, convex_hull_plot_2d
from folium import IFrame
from pyproj import Proj
from shapely.geometry import Polygon

In [2]:
# https://alysivji.github.io/getting-started-with-folium.html
# https://opendata.dc.gov/datasets/294e062cdf2c48d5b9cbc374d9709bc0_2/data

In [3]:
# save them to csvs
bikeData = pd.read_csv("data/bikes.csv")
bikePedData = pd.read_csv("data/bikePedData.csv")
bikeVehData = pd.read_csv("data/bikeVehData.csv")
streets = pd.read_csv("data/streets.csv")
streetSegs = pd.read_csv("data/streetSegs.csv")

In [4]:
print(list(bikeData))

['Unnamed: 0', 'TOTAL_BICYCLES', 'TOTAL_VEHICLES', 'TOTAL_PEDESTRIANS', 'LATITUDE', 'LONGITUDE', 'XCOORD', 'YCOORD', 'FATAL_BICYCLIST', 'MPDLATITUDE', 'MPDLONGITUDE', 'FROMDATE', 'STREETSEGID', 'ROUTEID', 'NEARESTINTSTREETNAME', 'NEARESTINTROUTEID', 'OFFINTERSECTION']


In [5]:
print(list(streets))

['OBJECTID', 'FACILITYID', 'STREETSEGID', 'SOURCEID', 'BIKELANELENGTH', 'FACILITY', 'PROPOSEDCYCLETRACK', 'Shape_Length', 'TRAVELDIRECTION', 'NOTES', 'BIKELANE_YEAR', 'PLANSREADY', 'GAP', 'GAP_NOTES', 'NEED_SYMBOL', 'NEED_SYM_1', 'REPAINT_LINE', 'YEAR_INSTALLED']


In [6]:
print(list(streetSegs))

['STREETSEGID', 'FACILITYID', 'SOURCEID', 'STREETID', 'REGISTEREDNAME', 'STREETTYPE', 'QUADRANT', 'DIRECTIONALITY', 'SEGMENTTYPE', 'FROMNODEID', 'TONODEID', 'FROMADDRESSLEFTTHEO', 'TOADDRESSLEFTTHEO', 'FROMADDRESSRIGHTTHEO', 'TOADDRESSRIGHTTHEO', 'BEGINMEASURE', 'ENDMEASURE', 'UPDATETIMESTAMP', 'OBJECTID_1', 'OBJECTID', 'SHAPELEN']


In [7]:
streets.FACILITY.value_counts()

Existing Bike Lane      908
Shared Lane             258
Cycle Track              88
Climbing Lane            62
Contraflow Bike Lane     45
                          5
Bus/Bike Lane             4
Name: FACILITY, dtype: int64

In [8]:
colorsList = ['red', 'blue', 'black', 'purple', 'orange', 'pink', 'green']

def getColor(ind):
    colorLen = len(colorsList)
    colorNum = ind%colorLen
    color = colorsList[colorNum]
    return color

def runCluster(dataset, eps, minSamples):
    accPoints = np.empty((0, 2))   
    
    for lat,long in zip(dataset['LATITUDE'], dataset['LONGITUDE']):
        accPoints = np.append(accPoints, np.array([[lat,long]]), axis=0)

    # Compute DBSCAN
    db = DBSCAN(eps=eps, min_samples=minSamples).fit(accPoints)

    clusters = db.fit_predict(accPoints)
    dataset['CLUSTER'] = clusters
    

    clusters = dataset['CLUSTER'].unique()   
    clusterDict = {}
    primaryStreetDict = {}
    secondaryStreetDict = {}
    primaryFacilityTypeDict = {}
    secondaryFacilityTypeDict = {}
    primaryStreetSegIdDict = {}
    secondaryStreetSegIdDict = {}
    
    for i in clusters:
        clusterSet = dataset[(dataset["CLUSTER"] == i) & (dataset["STREETSEGID"] > 0)]
        valueCounts = clusterSet['STREETSEGID'].value_counts()
        if (len(valueCounts.index) > 1):
            primaryValueId = valueCounts.index[0]
            primaryValue = streetSegs[streetSegs["STREETSEGID"] == int(primaryValueId)].reset_index()["REGISTEREDNAME"][0]
            secondaryValueId = valueCounts.index[1]
            secondaryValue = streetSegs[streetSegs["STREETSEGID"] == int(secondaryValueId)].reset_index()["REGISTEREDNAME"][0]
            primaryStreetDict[i] = primaryValue
            secondaryStreetDict[i] = secondaryValue
            
            facilityResp = streets[streets["STREETSEGID"] == int(primaryValueId)]
            if (len(facilityResp) > 0):    
                primaryFacilityTypeDict[i] = facilityResp.reset_index()["FACILITY"][0]
                primaryStreetSegIdDict[i] = primaryValueId
            else:
                primaryFacilityTypeDict[i] = "None"
                primaryStreetSegIdDict[i] = "None"
            facilityResp = streets[streets["STREETSEGID"] == int(secondaryValueId)]
            if (len(facilityResp) > 0):  
                secondaryFacilityTypeDict[i] = facilityResp.reset_index()["FACILITY"][0]
                secondaryStreetSegIdDict[i] = secondaryValueId
            else:
                secondaryFacilityTypeDict[i] = "None"
                secondaryStreetSegIdDict[i] = "None"
        elif (len(valueCounts.index) > 0):
            primaryValueId = valueCounts.index[0]
            primaryValue = streetSegs[streetSegs["STREETSEGID"] == int(primaryValueId)].reset_index()["REGISTEREDNAME"][0]
            primaryStreetDict[i] = primaryValue
            facilityResp = streets[streets["STREETSEGID"] == int(primaryValueId)]
            if (len(facilityResp) > 0):    
                primaryFacilityTypeDict[i] = facilityResp.reset_index()["FACILITY"][0]
                primaryStreetSegIdDict[i] = primaryValueId
            else:
                primaryFacilityTypeDict[i] = "None"
                primaryStreetSegIdDict[i] = "None"
                
            secondaryStreetDict[i] = "None"
            secondaryFacilityTypeDict[i] = "None"
            secondaryStreetSegIdDict[i] = "None"
        else:
            primaryStreetDict[i] = "None"
            secondaryStreetDict[i] = "None"
            primaryFacilityTypeDict[i] = "None"
            secondaryFacilityTypeDict[i] = "None"
            primaryStreetSegIdDict[i] = "None"
            secondaryStreetSegIdDict[i] = "None"
            
    primaryStreets = []
    secondaryStreets = []
    primaryFacilityTypes = []
    secondaryFacilityTypes = []
    primaryStreetSegIds = []
    secondaryStreetSegIds = []

    for i in dataset["CLUSTER"]:
        if (primaryStreetDict[i] == "None"):
            primaryStreets.append("None")
            primaryFacilityTypes.append("None")
            primaryStreetSegIds.append("None")
        else:
            primaryStreets.append(primaryStreetDict[i])
            primaryFacilityTypes.append(primaryFacilityTypeDict[i])
            primaryStreetSegIds.append(primaryStreetSegIdDict[i])
        if (secondaryStreetDict[i] == "None"):
            secondaryStreets.append("None")
            secondaryFacilityTypes.append("None")
            secondaryStreetSegIds.append("None")
        else:
            secondaryStreets.append(secondaryStreetDict[i])
            secondaryFacilityTypes.append(secondaryFacilityTypeDict[i])
            secondaryStreetSegIds.append(secondaryStreetSegIdDict[i])
       
    dataset['CLUSTERPRIMARYSTREET'] = primaryStreets
    dataset['CLUSTERSECONDARYSTREET'] = secondaryStreets
    dataset['CLUSTERprimaryFACILITYTYPE'] = primaryFacilityTypes
    dataset['CLUSTERSECONDARYFACILITYTYPE'] = secondaryFacilityTypes
    dataset['CLUSTERPRIMARYSTREETSEGID'] = primaryStreetSegIds
    dataset['CLUSTERSECONDARYSTREETSEGID'] = secondaryStreetSegIds
    return dataset

def makeHulls(dataset):
    features = []
    clusterDict = {}
    primaryStreetDict = {}
    secondaryStreetDict = {}
    primaryFacilityTypeDict = {}
    secondaryFacilityTypeDict = {}
    primaryStreetSegIdDict = {}
    secondaryStreetSegIdDict = {}
    totalAccidentNumberDict = {}
    for lat, long, cluster, primaryStreet, secondaryStreet, primaryFaciltyType, secondaryFaciltyType, primaryStreetSegId, secondaryStreetSegId in zip(dataset['LATITUDE'],dataset['LONGITUDE'],dataset['CLUSTER'],dataset['CLUSTERPRIMARYSTREET'],dataset['CLUSTERSECONDARYSTREET'],dataset['CLUSTERprimaryFACILITYTYPE'],dataset['CLUSTERSECONDARYFACILITYTYPE'], dataset['CLUSTERPRIMARYSTREETSEGID'], dataset['CLUSTERSECONDARYSTREETSEGID']):
        if (cluster != -1):
            if (cluster in clusterDict.keys()):
                thisCluster = clusterDict[cluster]
                thisCluster.append([long,lat])
                clusterDict[cluster] = thisCluster
                totalAccidentNumberDict[cluster] +=1
            else:
                clusterDict[cluster] = [[long,lat]]
                primaryStreetDict[cluster] = str(primaryStreet)
                secondaryStreetDict[cluster] = str(secondaryStreet)
                primaryFacilityTypeDict[cluster] = str(primaryFaciltyType)
                secondaryFacilityTypeDict[cluster] = str(secondaryFaciltyType)
                primaryStreetSegIdDict[cluster] = str(primaryStreetSegId)
                secondaryStreetSegIdDict[cluster] = str(secondaryStreetSegId)
                totalAccidentNumberDict[cluster] = 0

    hulls = []
    clusters = []
    primaryStreets = []
    secondaryStreets = []
    primaryFacilityTypes = []
    secondaryFacilityTypes = []
    primaryStreetSegIds = []
    secondaryStreetSegIds = []
    totalAccidentNumbers = []
    clusterAreas = []
    for cluster in clusterDict.keys():
        clusters.append(cluster)
        thisHull = ConvexHull(clusterDict[cluster])
        hulls.append(thisHull)
        primaryStreets.append(primaryStreetDict[cluster])
        secondaryStreets.append(secondaryStreetDict[cluster])
        primaryFacilityTypes.append(primaryFacilityTypeDict[cluster])
        secondaryFacilityTypes.append(secondaryFacilityTypeDict[cluster])
        primaryStreetSegIds.append(primaryStreetSegIdDict[cluster])
        secondaryStreetSegIds.append(secondaryStreetSegIdDict[cluster])
        totalAccidentNumbers.append(totalAccidentNumberDict[cluster])
        clusterAreas.append(area(thisHull))
        
    clusters = pd.DataFrame(
            {
                "hull" : hulls,
                "cluster" : clusters,
                "primaryStreet" : primaryStreets,
                "secondaryStreet" : secondaryStreets,
                "primaryFacilityType" : primaryFacilityTypes, 
                "secondaryFacilityType" : secondaryFacilityTypes,
                "primaryStreetSegId" : primaryStreetSegIds,
                "secondaryStreetSegId" : secondaryStreetSegIds,
                "totalAccidents" : totalAccidentNumbers,
                "clusterArea" : clusterAreas
            })
        
    return clusters


def makeGeoJson(clusters):
    shapes = []

    for cluster, hull in zip(clusters['cluster'], clusters['hull']):
        outline = []
        for p in hull.vertices:
            outline.append(list(hull.points[p]))
        outline.append(list(hull.points[hull.vertices[0]]))
        shapes.append(outline)

    clusters["shape"] = shapes
    
    myGeoJson = {"type": "FeatureCollection","features":[]}   
    
    for index, row in clusters.iterrows():
        shape = row['shape']
        primaryStreet = row['primaryStreet']
        secondaryStreet = row['secondaryStreet']
        primaryFacilityType = row['primaryFacilityType']
        secondaryFaciltyType = row['secondaryFacilityType']
        primaryStreetSegId = row['primaryStreetSegId']
        secondaryStreetSegId = row['secondaryStreetSegId']
        totalAccidents = row['totalAccidents']
        clusterArea = row['clusterArea']
        thisFeature = {
            "type": "Feature",
            "properties": {
                "name": str(shape),
                "primaryStreet" : primaryStreet,
                "secondaryStreet" : secondaryStreet,
                "primaryFacilityType" : primaryFacilityType,
                "secondaryFaciltyType" : secondaryFaciltyType,
                "primaryStreetSegId" : primaryStreetSegId,
                "secondaryStreetSegId" : secondaryStreetSegId,
                "totalAccidents" : totalAccidents,
                "clusterArea" : clusterArea
             },
            "geometry" : {
                "type" : "Polygon",
                "coordinates" : [shape]
            }
        }
        myGeoJson['features'].append(thisFeature)

    geo_str = json.dumps(myGeoJson)
    return geo_str, clusters

def area(hull):
    verts = [hull._points[i] for i in hull._vertices]
    polygon = Polygon(verts)
    polyArea = 1000000 * polygon.area
    print(polyArea)
    return polyArea


def style_function(feature):
#     print(feature['properties'])
    # styling guide  https://python-visualization.github.io/folium/modules.html
    fillColor = ""
    if (feature['properties']['primaryFacilityType'] == "None"):
        return {
            'fillColor': 'red',
            'fillOpacity' : .5,
            'color' : 'black'
        }
    elif (feature['properties']['primaryFacilityType'] == "Shared Lane"):
        return {
            'fillColor': 'orange',
            'fillOpacity' : .5,
            'color' : 'black'
        }
    elif (feature['properties']['primaryFacilityType'] == "Contraflow Bike Lane"):
        return {
            'fillColor': 'purple',
            'fillOpacity' : .5,
            'color' : 'black'
        }
    elif (feature['properties']['primaryFacilityType'] == "Existing Bike Lane"):
        return {
            'fillColor': 'blue',
            'fillOpacity' : .5,
            'color' : 'black'
        }
    elif (feature['properties']['primaryFacilityType'] == "Climbing Lane"):
        return {
            'fillColor': 'pink',
            'fillOpacity' : .5,
            'color' : 'black'
        }
    elif (feature['properties']['primaryFacilityType'] == "Cycle Track"):
        return {
            'fillColor': 'green',
            'fillOpacity' : .5,
            'color' : 'black'
        }
    else:
        return {
            'fillColor': 'black',
            'fillOpacity' : .5,
            'color' : 'black'
        }

def mapAll(dataset):

    dcMap = folium.Map(location=[38.9072, -77.0369], zoom_start=13)

    featureGroupDict = {}
    featureGroupDict[1] = folium.FeatureGroup(name="bike fatalities")
    featureGroupDict[2] = folium.FeatureGroup(name="bike incident")
    counter = 0
    for lat, long, fb in zip(dataset['LATITUDE'], dataset['LONGITUDE'], dataset['FATAL_BICYCLIST']):
        if (fb > 0):
            folium.CircleMarker(location=[lat, long], radius = 2, color = 'red').add_to(featureGroupDict[1])
        else:
            folium.CircleMarker(location=[lat, long], radius = .5, color = 'black').add_to(featureGroupDict[2])
        
    for key in featureGroupDict.keys():
        dcMap.add_child(featureGroupDict[key])

    folium.map.LayerControl('topright', collapsed=False).add_to(dcMap)
    return dcMap

def makeMap(dataset):
    # plotting all bike accidents
    dcMap = folium.Map(location=[38.9072, -77.0369], zoom_start=13)

    featureGroupDict = {}
        
    counter = 0
    for lat, long, cluster, clusterprimaryStreet, clusterSecondaryStreet, fb in zip(dataset['LATITUDE'], dataset['LONGITUDE'], dataset["CLUSTER"], dataset["CLUSTERPRIMARYSTREET"], dataset["CLUSTERSECONDARYSTREET"], dataset['FATAL_BICYCLIST']):
        if (cluster != -1):
            if (cluster not in featureGroupDict.keys()):
                featureGroupDict[cluster] = folium.FeatureGroup(name=(str(clusterprimaryStreet) + " and " + str(clusterSecondaryStreet)))
            if (fb > 0):
                folium.CircleMarker(location=[lat, long], radius = 2, color = 'red').add_to(featureGroupDict[cluster])
            else:
                folium.CircleMarker(location=[lat, long], radius = .5, color = 'black').add_to(featureGroupDict[cluster])

        counter = counter + 1 
        
    clusters = makeHulls(dataset)
    geoString, clusters = makeGeoJson(clusters)
    featureGroupDict["Layers"] = folium.FeatureGroup(name="Layers")
    ourLayers = folium.GeoJson(json.loads(geoString), style_function=style_function,
                               tooltip=folium.features.GeoJsonTooltip(
                                fields=['primaryStreet','secondaryStreet','primaryFacilityType','secondaryFaciltyType','primaryStreetSegId','secondaryStreetSegId']
                              ))
    ourLayers.add_to(featureGroupDict["Layers"]);
    
    for key in featureGroupDict.keys():
        dcMap.add_child(featureGroupDict[key])
        
    folium.map.LayerControl('topright', collapsed=False).add_to(dcMap)

    item_txt = """<br> &nbsp; {item} &nbsp; <i class="fa fa-map-marker fa-2x" style="color:{col}"></i>"""
    html_itms = item_txt.format(item= "None" , col= "red")

    legend_html = """
     <div style="
     position: fixed; 
     bottom: 5px; left: 5px; width: 220px; height: 160px; 
     border:2px solid grey; z-index:9999; 
     background-color:white;
     opacity: .85;
     font-size:14px;
     font-weight: bold;
     ">
     <div style= "text-align:center">Street Bike Facilities</div>
     <br>
     <div><div style="margin-left:20px;">None </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:red;">  </div></div>
     <div><div style="margin-left:20px;">Shared Lane </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:orange;">  </div></div>
     <div><div style="margin-left:20px;">Contraflow Bike Lane </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:purple;">  </div></div>
     <div><div style="margin-left:20px;">Existing Lane </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:blue;">  </div></div>
     <div><div style="margin-left:20px;">Climbing Lane </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:pink;">  </div></div>
     <div><div style="margin-left:20px;">Cycle Track </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:green;">  </div></div>
      
      
      
      
      
      </div> """
    dcMap.get_root().html.add_child(folium.Element( legend_html ))
    
    framedData = pd.read_json(geoString)
    clusters.to_csv(str(len(framedData)) + "clustersdata.csv")
    return dcMap

In [9]:
# NOTES

# bikeData
# worst 3 hubs runCluster(bikeData, 0.0015, 25)
# more hubs runCluster(bikeData, 0.0014, 15)
# tightly packed runCluster(bikeData, 0.0005, 7)

# bikePedData
# runCluster(bikePedData, 0.0030, 3)

# bikeVehData
# bikeVehData(bikeVehData, 0.0010, 10)

# bikePedData

# clusteredData = runCluster(bikePedData, 0.01, 1)
# dcMap = makeMap(clusteredData)
# dcMap

# # bike vehicle data

# clusteredData = runCluster(bikeVehData, 0.0010, 10)
# dcMap = makeMap(clusteredData)
# dcMap

In [10]:
dcMap = mapAll(bikeData)
dcMap

In [11]:
clusteredData = runCluster(bikeData, 0.0015, 25)
dcMap = makeMap(clusteredData)
dcMap

3.5729474999967734
4.631516499978574
7.235664999925086
9.849336000010553


In [12]:
clusteredData = runCluster(bikeData, 0.0014, 15)
dcMap = makeMap(clusteredData)
dcMap

17.48424399998424
5.84407200003003
2.077987000024997
6.439279999948604
9.1393130000465
2.707942000046976
7.971774999999311
11.18083649997223
8.488437999982382
3.8638910000191258
4.483600499980776
1.6949989999898025
6.889772999994117


In [13]:
clusteredData = runCluster(bikeData, 0.0005, 7)
dcMap = makeMap(clusteredData)
dcMap

0.013003999998107323
0.04443899999688539
0.11651149999964405
0.12435299999776743
0.09387800000116553
0.5876744999953928
0.12981849999852707
0.19644599999830514
2.9999999919568835e-06
0.10871699999859037
0.26306000000323887
0.10095149999962136
0.141216000002828
0.043717500001548254
8.95000029018304e-05
0.10999199999802133
0.08318499999793645
